# Wildfires

Load the fire and wind data on Aug. 5th saved in csv file. the fire data is every 10 mins. The wind data is every 60 mins.

Save the generated fire data in 2D matrices under fire_grid directory
Save the generated wind data in 2D matrices under wind_interp directory. The data is interpolated to 10 mins.

In [1]:
FIRE_DF_PATH = "fire_df_08_05.csv"
WIND_DF_PATH = "wind_df_08_05.csv"

from timeinterp import *

time_interp(data_path=WIND_DF_PATH)

from preprocessing import *
preprocess(data_path=FIRE_DF_PATH)

from cross_correlation import *


Current working directory: /Users/timothyliu/PycharmProjects/wildfires
[35.83098754882815, 35.83098754882815, 35.83098754882815, 34.62005004882815, 34.62005004882815] ... [33.04973754882815, 33.04973754882815, 34.05755004882815, 34.05755004882815, 34.05755004882815]
[2.51220703125, 2.51220703125, 2.51220703125, 3.186279296875, 3.186279296875] ... [3.5322265625, 3.5322265625, 1.92431640625, 1.92431640625, 1.92431640625]
[2.1080322265625, 2.1080322265625, 2.1080322265625, 0.2218017578125, 0.2218017578125] ... [0.7471923828125, 0.7471923828125, 0.5323486328125, 0.5323486328125, 0.5323486328125]
[-121.49714217716529, -121.4791758714829, -121.4612095658005, -121.44324326011812, -121.42527695443573] ... [-120.58086058736338, -120.56289428168098, -120.54492797599859, -120.52696167031621, -120.50899536463382]
[39.516889348417756, 39.516889348417756, 39.516889348417756, 39.516889348417756, 39.516889348417756] ... [40.48706985526684, 40.48706985526684, 40.48706985526684, 40.48706985526684, 40.48

/Users/timothyliu/PycharmProjects/wildfires/cross_correlation.py:42: RuntimeWarning: invalid value encountered in divide
  norm_grid = np.nan_to_num(sum_grid / divide_grid)       # nan = divide by 0 (no correlation)


Lag: 20
Cutoff: (24, 4)
Lag: 30
Cutoff: (24, 4)
Lag: 40
Cutoff: (24, 4)
Lag: 50
Cutoff: (24, 4)
Lag: 60
Cutoff: (24, 4)
Lag: 70
Cutoff: (24, 4)
Lag: 80
Cutoff: (24, 4)
Lag: 90
Cutoff: (24, 4)
Lag: 100
Cutoff: (24, 4)
Lag: 110
Cutoff: (24, 4)
